In [26]:
# Calculate Distance - Notebook
# Author: S. Chu
# Date: 2024-10-27

# Setup:
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import folium

geolocator = Nominatim(user_agent="Hello World")

# Define set of cities:
cities = [
    "Asheville, NC",
    "San Diego, CA",
    "Pearl Harbor, HI",
    "Munich, Germany",
    "Yokohama, Japan",
    "London, England",
    "Groton, CT"
]

# Target location (Coordinate):
target_location = (38.9072, -77.0369)

# Functions
# Target Location or Coordinate:
def is_coordinate(location):
    if isinstance(location, tuple) and len(location) == 2:
        latitude, longitude = location
        if isinstance(latitude, (int,float)) and isinstance(longitude, (int,float)):
            if -90 <= latitude <= 90 and -180 <= longitude <= 180:
                return True
    return False

# Reverse coordinate search:
def reverse_coordinate_search(target_location):
    location = geolocator.reverse(target_location)
    return location

# Find location coordinate:
def get_coordinate(target_location):
    location = (
        round(target_location.latitude, 6),
        round(target_location.longitude, 6)
        )
    return location

# Calculate distance (city, distance from target):
def calculate_distances(cities, target_location):
    distances = {}
    for city in cities:
        location = geolocator.geocode(city)
        if location:
            city_coordinate = (location.latitude, location.longitude)
            distance = geodesic(city_coordinate, target_location).kilometers
            distances[city] = round(distance, 1)
        else:
            distances[city] = None
    return distances

# Get coordinates (city, coordinates):
def get_coordinates(cities, target_location):
    coordinates = {}
    for city in cities:
        location = geolocator.geocode(city)
        if location:
            city_coordinate = (
                round(location.latitude, 6),
                round(location.longitude, 6)
            )
            coordinates[city] = city_coordinate
        else:
            coordinates[city] = None
    return coordinates

# Get data and sort (city, coordinates, distance from target):
def sorted_distances(cities, target_location):
    distances = {}
    for city in cities:
        location = geolocator.geocode(city)
        if location:
            city_coordinate = (
                round(location.latitude, 6),
                round(location.longitude, 6)
            )
            distance = geodesic(city_coordinate, target_location).kilometers
            distances[city] = (city, city_coordinate, round(distance, 1))
        else:
            distances[city] = None
            distances[city] = (city, 'N/A', 'N/A')
    sorted_distances = sorted(
        distances.values(), key=lambda x: x[2]
        if x[2] != 'N/A' else float('inf')      # Substitutes 'N/A' as 'inf'
        )
    print_data(sorted_distances)
    return sorted_distances

# Print sorted distances - Used with sorted_distances:
def print_data(sorted_data):
    print("\nSorted Distances:")
    print(f"{'City':<25} {'Coordinates':<30} {'Distance (km)':<15}")  # Header
    for city, coord, dist in sorted_data:
        if dist != 'N/A':
            formatted_coordinates = f"({coord[0]}, {coord[1]})"
            print(f"{city:<25} {formatted_coordinates:<30} {dist:>10.1f} km")
        else:
            print(f"{city:<25} {'N/A':<30} {'N/A':>10}")
    return sorted_data


In [27]:
# Call function
distances = calculate_distances(cities, target_location)
coordinates = get_coordinates(cities, target_location)
output = sorted_distances(cities, target_location)



Sorted Distances:
City                      Coordinates                    Distance (km)  
Groton, CT                (41.35016, -72.0762)                502.2 km
Asheville, NC             (35.595363, -82.550841)             611.7 km
San Diego, CA             (32.71742, -117.162772)            3660.7 km
London, England           (51.507446, -0.127765)             5912.9 km
Munich, Germany           (48.137108, 11.575382)             6833.3 km
Pearl Harbor, HI          (21.355775, -157.999149)           7792.7 km
Yokohama, Japan           (35.444395, 139.636773)           10953.3 km


In [28]:
# Call reverse_coordinate_search function
target = reverse_coordinate_search(target_location)
print(f"{'Target Location:':<20} {target}")
target = get_coordinate(target)
print(f"{'Target Coordinate:':<20} {target}")

Target Location:     Scott Circle Northwest, Ward 2, Washington, District of Columbia, 20036, United States
Target Coordinate:   (38.90717, -77.036925)


In [29]:
# Create map with distances:
def map_distances(cities, target_location):
    distances = {}

    # Initalize map centered on target location
    m = folium.Map(location=target_location, zoom_start=5)
    folium.Marker(
        location = target_location,
        popup = f"{target_location}",
        icon = folium.Icon(color="red")
    ).add_to(m)

    # Calculate distances and mark on map
    for city in cities:
        location = geolocator.geocode(city)
        if location:
            city_coordinate = (location.latitude, location.longitude)
            distance = geodesic(city_coordinate, target_location).kilometers
            distances[city] = (city_coordinate, round(distance, 1))

            # Add city marker on map
            folium.Marker(
                location = city_coordinate,
                popup = f"{city}: {distances[city][1]} km",
                icon = folium.Icon(color="blue")
            ).add_to(m)

            # Draw line between target location and city on map
            folium.PolyLine(
                [target_location, city_coordinate],
                color = "blue",
                weight = 2,
                opacity = 0.5
            ).add_to(m)

        else:
            distances[city] = ('N/A', 'N/A')

    return m, distances

In [30]:
# Generate the map
map_obj, distances = map_distances(cities, target_location)

# Display the map
map_obj